In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

touch_values = []
classi = []

# === 1. Parse touch_prediction.txt ===
def load_touch_predictions(filepath):
    """Loads the touch prediction file and extracts the arrays."""
    values = []
    with open(filepath, 'r') as f:
        for line in f:
            # Remove "Step x:" prefix and convert to list of floats
            clean_line = line.split(':', 1)[-1].strip()
            arr = eval(clean_line)  # safely parse the list
            values.append(arr)
    return np.array(values)  # shape (T, 8)

# === 2. Main code to loop through datasets ===
results_dir = "Results"  # Path to the directory with multiple datasets
for d in os.listdir(results_dir):
    # Construct full path to touch predictions and labels for each dataset
    touch_file = os.path.join(results_dir, d, "touch_predictions.txt")
    label_file = os.path.join(results_dir, d, "touch_type.txt")
    
    if os.path.exists(touch_file) and os.path.exists(label_file):
        print(f"Processing dataset: {d}")

        # Load touch prediction arrays (shape = (T, 8))
        X_seq = load_touch_predictions(touch_file)
        #print (X_seq)
        X_flat = X_seq.flatten()  # Flatten to (T*8,)
        
        # Load the label (assumed to be one per dataset)
        with open(label_file, "r") as f:
            y_raw = f.read().strip()
        
        touch_values.append(X_flat)
        classi.append(y_raw)
    else:
        print(f"Skipping dataset {d} (missing files).")

# Encode the label (Pain/Touch -> 0/1)
le = LabelEncoder()
y_encoded = le.fit_transform(classi)

# Train classifier (SVM in this case)
clf = SVC(kernel='linear')
clf.fit(touch_values, y_encoded)  # Train with one sample per dataset

# Predict
pred = clf.predict([X_flat])
print(f"Predicted: {le.inverse_transform(pred)[0]} for {d}")



Processing dataset: test_29
Processing dataset: test_16
Processing dataset: test_38
Processing dataset: test_37
Processing dataset: test_41
Processing dataset: test_15
Processing dataset: test_18
Processing dataset: test_25
Processing dataset: test_31
Processing dataset: test_23
Processing dataset: test_11
Processing dataset: test_8
Processing dataset: test_14
Processing dataset: test_28
Processing dataset: test_33
Processing dataset: test_12
Processing dataset: test_5
Processing dataset: test_19
Processing dataset: test_6
Processing dataset: test_35
Processing dataset: test_21
Processing dataset: test_27
Processing dataset: test_3
Processing dataset: test_7
Processing dataset: test_44
Processing dataset: test_36
Processing dataset: test_34
Processing dataset: test_1
Processing dataset: test_17
Processing dataset: test_42
Processing dataset: test_20
Processing dataset: test_10
Processing dataset: test_40
Processing dataset: test_24
Processing dataset: test_43
Processing dataset: test_4

In [22]:
pred = clf.predict([touch_values[0]])
print(f"Predicted: {le.inverse_transform(pred)[0]}")

Predicted: PAIN


In [18]:
# Perform cross-validation
scores = cross_val_score(clf, touch_values, y_encoded, cv=5)

# Print the cross-validation results
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean():.4f}")

Cross-validation scores: [1.         1.         1.         0.88888889 1.        ]
Mean cross-validation score: 0.9778


In [23]:
import pickle
with open("svm_model.pkl", "wb") as model_file:
    pickle.dump(clf, model_file)
    pickle.dump(le, model_file)  # Save the label encoder as well

print("Model and label encoder saved to 'svm_model.pkl'.")

Model and label encoder saved to 'svm_model.pkl'.
